In [3]:
# PNG이미지를 바이너리 마스크로 변경해서 폴더에 저장

In [4]:
from PIL import Image
import numpy as np
import os

# PNG 마스크가 있는 폴더와 변환 후 바이너리 마스크 저장 폴더
INPUT_MASK_DIR = "./png_masks"       # 기존 PNG 마스크가 있는 폴더
OUTPUT_MASK_DIR = "./binary_masks"  # 생성된 바이너리 마스크를 저장할 폴더
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)  # 출력 폴더 생성

def create_binary_masks(input_dir, output_dir):
    """PNG 마스크를 바이너리 (.npy) 마스크로 변환"""
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".png"):  # PNG 파일만 처리
            input_path = os.path.join(input_dir, file_name)
            output_path = os.path.join(output_dir, file_name.replace(".png", ".npy"))

            # PNG 마스크 읽기
            mask = Image.open(input_path).convert("L")  # 흑백 모드 (L)로 변환
            mask_array = np.array(mask)

            # 흑백 값을 바이너리 값 (0, 1)로 변환
            binary_mask = (mask_array > 0).astype(np.uint8)  # 0 아닌 모든 픽셀을 1로

            # .npy 파일로 저장
            np.save(output_path, binary_mask)
            print(f"Saved binary mask: {output_path}")

# PNG 마스크를 모두 변환 실행
create_binary_masks(INPUT_MASK_DIR, OUTPUT_MASK_DIR)
print(f"바이너리 마스크들이 '{OUTPUT_MASK_DIR}'에 저장되었습니다.")


Saved binary mask: ./binary_masks\1.npy
Saved binary mask: ./binary_masks\10.npy
Saved binary mask: ./binary_masks\11.npy
Saved binary mask: ./binary_masks\13.npy
Saved binary mask: ./binary_masks\14.npy
Saved binary mask: ./binary_masks\15.npy
Saved binary mask: ./binary_masks\16.npy
Saved binary mask: ./binary_masks\17.npy
Saved binary mask: ./binary_masks\18.npy
Saved binary mask: ./binary_masks\19.npy
Saved binary mask: ./binary_masks\2.npy
Saved binary mask: ./binary_masks\20.npy
Saved binary mask: ./binary_masks\21.npy
Saved binary mask: ./binary_masks\22.npy
Saved binary mask: ./binary_masks\23.npy
Saved binary mask: ./binary_masks\24.npy
Saved binary mask: ./binary_masks\25.npy
Saved binary mask: ./binary_masks\26.npy
Saved binary mask: ./binary_masks\27.npy
Saved binary mask: ./binary_masks\29.npy
Saved binary mask: ./binary_masks\3.npy
Saved binary mask: ./binary_masks\30.npy
Saved binary mask: ./binary_masks\31.npy
Saved binary mask: ./binary_masks\32.npy
Saved binary mask: 

# 이미지와 마스크를 TFR레코드로 변환환

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

def maintain_aspect_ratio_with_padding(image, mask, target_height=624, target_width=288):
    """종횡비를 유지하면서 패딩을 추가하는 함수"""
    
    # 원본 크기
    orig_h, orig_w = image.size[1], image.size[0]
    
    # 타겟 크기에 맞추기 위한 스케일 계산
    scale_h = target_height / orig_h
    scale_w = target_width / orig_w
    scale = min(scale_h, scale_w)  # 더 작은 스케일 사용
    
    # 새로운 크기 계산
    new_h = int(orig_h * scale)
    new_w = int(orig_w * scale)
    
    # 이미지 리사이즈
    image = image.resize((new_w, new_h), Image.Resampling.LANCZOS)
    mask = mask.resize((new_w, new_h), Image.Resampling.NEAREST)
    
    # 새로운 이미지 생성 (패딩 포함)
    padded_image = Image.new('RGB', (target_width, target_height), (0, 0, 0))
    padded_mask = Image.new('L', (target_width, target_height), 0)
    
    # 중앙에 이미지 배치
    paste_x = (target_width - new_w) // 2
    paste_y = (target_height - new_h) // 2
    
    padded_image.paste(image, (paste_x, paste_y))
    padded_mask.paste(mask, (paste_x, paste_y))
    
    return padded_image, padded_mask

def create_tfrecord(image_dir, mask_dir, output_path):
    """TFRecord 생성"""
    writer = tf.io.TFRecordWriter(output_path)
    
    print("TFRecord 생성 시작...")
    count = 0
    
    for image_name in os.listdir(image_dir):
        if not image_name.endswith(('.jpg', '.jpeg', '.png')):
            continue
            
        # 파일 경로
        image_path = os.path.join(image_dir, image_name)
        mask_name = os.path.splitext(image_name)[0] + '.png'
        mask_path = os.path.join(mask_dir, mask_name)
        
        if not os.path.exists(mask_path):
            print(f"마스크 파일을 찾을 수 없음: {mask_name}")
            continue
        
        try:
            # 이미지와 마스크 로드
            image = Image.open(image_path).convert('RGB')
            mask = Image.open(mask_path).convert('L')
            
            # 종횡비 유지하면서 리사이즈 및 패딩
            image, mask = maintain_aspect_ratio_with_padding(image, mask)
            
            # numpy 배열로 변환
            image_array = np.array(image)
            mask_array = np.array(mask)
            
            # 이미지를 바이트로 인코딩
            image_bytes = tf.io.encode_jpeg(image_array).numpy()
            mask_bytes = tf.io.encode_png(mask_array[:, :, np.newaxis]).numpy()
            
            # TFRecord 특성 정의
            feature = {
                'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
                'image/segmentation/class/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[mask_bytes]))
            }
            
            # 예제 생성 및 작성
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
            
            count += 1
            if count % 10 == 0:
                print(f"{count}개의 이미지 처리 완료")
                
        except Exception as e:
            print(f"오류 발생 ({image_name}): {str(e)}")
            continue
    
    writer.close()
    print(f"총 {count}개의 이미지를 처리했습니다.")
    print(f"TFRecord 생성 완료: {output_path}")

# 실행 코드
IMAGE_DIR = './input_images'
MASK_DIR = './binary_masks'
OUTPUT_TFRECORD_DIR = './tfrecords'
os.makedirs(OUTPUT_TFRECORD_DIR, exist_ok=True)

tfrecord_path = os.path.join(OUTPUT_TFRECORD_DIR, 'train.tfrecord')
create_tfrecord(IMAGE_DIR, MASK_DIR, tfrecord_path)

# TFRecord 확인용 코드
# 
#
def create_tfrecord(image_dir, mask_dir, output_path):
    writer = tf.io.TFRecordWriter(output_path)
    print("TFRecord 생성 시작...")
    count = 0
    
    # 이미지 파일 목록 가져오기
    image_files = sorted(os.listdir(image_dir))
    mask_files = sorted(os.listdir(mask_dir))
    
    print(f"발견된 이미지 파일: {len(image_files)}")
    print(f"발견된 마스크 파일: {len(mask_files)}")
    
    for image_name in image_files:
        if not image_name.endswith(('.jpg', '.jpeg', '.png')):
            continue
        
        # 파일 경로
        image_path = os.path.join(image_dir, image_name)
        mask_name = image_name  # 확장자 변환 제거
        mask_path = os.path.join(mask_dir, mask_name)
        
        if not os.path.exists(mask_path):
            print(f"마스크 파일을 찾을 수 없음: {mask_name}")
            # 다른 가능한 마스크 파일명 시도
            mask_name_without_ext = os.path.splitext(image_name)[0]
            possible_mask_names = [
                f"{mask_name_without_ext}.png",
                f"{mask_name_without_ext}.npy",
                f"{mask_name_without_ext}_mask.png",
                f"{mask_name_without_ext}_mask.npy"
            ]
            
            found = False
            for possible_name in possible_mask_names:
                possible_path = os.path.join(mask_dir, possible_name)
                if os.path.exists(possible_path):
                    mask_path = possible_path
                    found = True
                    print(f"대체 마스크 파일 찾음: {possible_name}")
                    break
            
            if not found:
                continue


# TFRecord 확인
check_tfrecord(tfrecord_path)


TFRecord 생성 시작...
마스크 파일을 찾을 수 없음: 1.png
마스크 파일을 찾을 수 없음: 10.png
마스크 파일을 찾을 수 없음: 11.png
마스크 파일을 찾을 수 없음: 13.png
마스크 파일을 찾을 수 없음: 14.png
마스크 파일을 찾을 수 없음: 15.png
마스크 파일을 찾을 수 없음: 16.png
마스크 파일을 찾을 수 없음: 17.png
마스크 파일을 찾을 수 없음: 18.png
마스크 파일을 찾을 수 없음: 19.png
마스크 파일을 찾을 수 없음: 2.png
마스크 파일을 찾을 수 없음: 20.png
마스크 파일을 찾을 수 없음: 21.png
마스크 파일을 찾을 수 없음: 22.png
마스크 파일을 찾을 수 없음: 23.png
마스크 파일을 찾을 수 없음: 24.png
마스크 파일을 찾을 수 없음: 25.png
마스크 파일을 찾을 수 없음: 26.png
마스크 파일을 찾을 수 없음: 27.png
마스크 파일을 찾을 수 없음: 29.png
마스크 파일을 찾을 수 없음: 3.png
마스크 파일을 찾을 수 없음: 30.png
마스크 파일을 찾을 수 없음: 31.png
마스크 파일을 찾을 수 없음: 32.png
마스크 파일을 찾을 수 없음: 6.png
마스크 파일을 찾을 수 없음: 7.png
마스크 파일을 찾을 수 없음: 8.png
마스크 파일을 찾을 수 없음: 9.png
총 0개의 이미지를 처리했습니다.
TFRecord 생성 완료: ./tfrecords\train.tfrecord


# 딥랩 학습 스크립트!!